In [2]:
import time
import os
#import tensorflow as tf
import inspect

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import keras.applications as kapp

print(keras.__version__)

Using plaidml.keras.backend backend.


2.2.4


In [22]:
dir(keras.metrics)

['K',
 'MAE',
 'MAPE',
 'MSE',
 'MSLE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'binary_accuracy',
 'binary_crossentropy',
 'categorical_accuracy',
 'categorical_crossentropy',
 'cosine',
 'cosine_proximity',
 'deserialize',
 'deserialize_keras_object',
 'division',
 'get',
 'hinge',
 'kullback_leibler_divergence',
 'logcosh',
 'mae',
 'mape',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_logarithmic_error',
 'mse',
 'msle',
 'poisson',
 'print_function',
 'serialize',
 'serialize_keras_object',
 'six',
 'sparse_categorical_accuracy',
 'sparse_categorical_crossentropy',
 'sparse_top_k_categorical_accuracy',
 'squared_hinge',
 'top_k_categorical_accuracy']

In [3]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )
    return model

batch_size = 32
def get_dataset():
    num_val_samples = 10000

    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        (x_train, y_train),
        (x_val, y_val),
        (x_test, y_test)
    )

In [4]:
model = get_compiled_model()
train_dataset, val_dataset, test_dataset = get_dataset()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


11493376/11490434 [==============================] - 2s 0us/step


In [32]:
inspect.getargspec(model.fit)

c:\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  """Entry point for launching an IPython kernel.


ArgSpec(args=['self', 'x', 'y', 'batch_size', 'epochs', 'verbose', 'callbacks', 'validation_split', 'validation_data', 'shuffle', 'class_weight', 'sample_weight', 'initial_epoch', 'steps_per_epoch', 'validation_steps'], varargs=None, keywords='kwargs', defaults=(None, None, None, 1, 1, None, 0.0, None, True, None, None, 0, None, None))

In [6]:
start = time.time()
# Train the model on all available devices.
model.fit(x = train_dataset[0], y = train_dataset[1], batch_size = batch_size, epochs=10, validation_data=val_dataset)
print("Ran in {} seconds".format(time.time() - start))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 49s 972us/step - loss: 9.8153 - sparse_categorical_accuracy: 0.1109 - val_loss: 9.8546 - val_sparse_categorical_accuracy: 0.1120
Epoch 2/10
50000/50000 [==============================] - 44s 876us/step - loss: 9.8153 - sparse_categorical_accuracy: 0.1109 - val_loss: 9.8546 - val_sparse_categorical_accuracy: 0.1120
Epoch 3/10
50000/50000 [==============================] - 43s 866us/step - loss: 9.8153 - sparse_categorical_accuracy: 0.1109 - val_loss: 9.8546 - val_sparse_categorical_accuracy: 0.1120
Epoch 4/10
50000/50000 [==============================] - 107s 2ms/step - loss: 9.8153 - sparse_categorical_accuracy: 0.1109 - val_loss: 9.8546 - val_sparse_categorical_accuracy: 0.1120
Epoch 5/10
50000/50000 [==============================] - 59s 1ms/step - loss: 9.8153 - sparse_categorical_accuracy: 0.1109 - val_loss: 9.8546 - val_sparse_categorical_accuracy: 0.1120
Epoch 6/10
50000/5

RTX 2060 time: 130.99 seconds
RX 470 time: 505.87 seconds

In [15]:
inspect.getargspec(model.evaluate)

c:\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  """Entry point for launching an IPython kernel.


ArgSpec(args=['self', 'x', 'y', 'batch_size', 'verbose', 'sample_weight', 'steps'], varargs=None, keywords=None, defaults=(None, None, None, 1, None, None))

In [21]:
# Test the model on all available devices.
results = model.evaluate(x = test_dataset[0], y = test_dataset[1])
print("test loss, test acc:", results)

10000/10000 [==============================] - 1s 86us/step
test loss, test acc: [11.392269830322265, 0.2026]
